In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


c_id = "485d4ce5cc2948f5abbfd3c9271677f9"
c_se ="7dfca8010d314470b1d3d641c52052d8"

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=c_id, client_secret=c_se))

In [2]:
import config

In [21]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd


# The "sp" variable has two useful funtions:
# The first usefull function is:
# .search(q='',limit=n)
# .search(q="track:"+song_name+" artist:"+artist_name,limit=5) to restrict to a song name and artist.
# Where the "q" keyword is the query you want to perform on spotify: song_name, artist,...
# while The "limit" keyword will limit the number of returned results.
#
# The second usefull function is:
# .audio_features([URL|URI|ID])
# which returns some 'features of the song', that after cleanup, we can use in order to characterize a song.

results = sp.search(q="Billie Eilish",limit=1,market="GB")
results
#json_results = json.dumps(results, ensure_ascii=True)
#json_results
#results['tracks']['items'][0]['external_urls']["spotify"]

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Billie+Eilish&type=track&market=GB&offset=0&limit=1',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6qqNVTkY8uBg9cP3Jd7DAH'},
       'href': 'https://api.spotify.com/v1/artists/6qqNVTkY8uBg9cP3Jd7DAH',
       'id': '6qqNVTkY8uBg9cP3Jd7DAH',
       'name': 'Billie Eilish',
       'type': 'artist',
       'uri': 'spotify:artist:6qqNVTkY8uBg9cP3Jd7DAH'}],
     'external_urls': {'spotify': 'https://open.spotify.com/album/0JGOiO34nwfUdDrD612dOp'},
     'href': 'https://api.spotify.com/v1/albums/0JGOiO34nwfUdDrD612dOp',
     'id': '0JGOiO34nwfUdDrD612dOp',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b2732a038d3bf875d23e4aeaa84e',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e022a038d3bf875d23e4aeaa84e',
       'width': 300},
      {'height': 64,
       'url': 'http

In [22]:
results["tracks"]["items"][0]["external_urls"]["spotify"]

'https://open.spotify.com/track/4RVwu0g32PAqgUiJoXsdF8'

In [23]:
track_id=results['tracks']['items'][0]['external_urls']["spotify"][31:]
track_id

'4RVwu0g32PAqgUiJoXsdF8'

In [41]:
results['tracks'].keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [24]:
from IPython.display import IFrame

#track_id = "1rfORa9iYmocEsnnZGMVC4"

IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [25]:
results.keys()

dict_keys(['tracks'])

In [26]:
results['tracks'].keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [63]:
def call_playlist(username, playlist_id):
    results = sp.user_playlist(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
    
    #step1

    playlist_features_list = ["artist","album","track_name","track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    #step2
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)

    #Step 3
        
        
    return playlist_df

In [64]:
call_playlist("spotify","5S8SJdl1BDc0ugpkEvFsIL")

KeyError: 'items'

In [53]:
import pprint
pprint.pprint(results)

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Billie+Eilish&type=track&market=GB&offset=0&limit=1',
            'items': [{'album': {'album_type': 'album',
                                 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6qqNVTkY8uBg9cP3Jd7DAH'},
                                              'href': 'https://api.spotify.com/v1/artists/6qqNVTkY8uBg9cP3Jd7DAH',
                                              'id': '6qqNVTkY8uBg9cP3Jd7DAH',
                                              'name': 'Billie Eilish',
                                              'type': 'artist',
                                              'uri': 'spotify:artist:6qqNVTkY8uBg9cP3Jd7DAH'}],
                                 'external_urls': {'spotify': 'https://open.spotify.com/album/0JGOiO34nwfUdDrD612dOp'},
                                 'href': 'https://api.spotify.com/v1/albums/0JGOiO34nwfUdDrD612dOp',
                                 'id': '0JGOiO34nw

In [54]:
def get_playlist_tracks(creator, playlist_id):
    results = sp.user_playlist_tracks(creator,playlist_id,market="GB")
    tracks = playlist_df['items']
    while results['next']:
        playlist_df = sp.next(playlist_df)
        tracks.extend(playlist_df['items'])
    return tracks

In [55]:
len(call_playlist("spotify","5S8SJdl1BDc0ugpkEvFsIL"))

100